In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv("Input/asa_predictions_combined.csv", sep=';', header=0)
df2 = pd.read_csv("Input/ac_predictions_combined.csv", sep=';', header=0)

In [ ]:
df.head(5)

# Hard Voting

In [ ]:
df.head(1)

In [5]:
hvoting = df.copy()
for ind, i in enumerate(hvoting.CNN_Predictions):
    for j, n in enumerate(i):
        if n >= 0.5:
            i[j] = 1
        else:
            i[j] = 0

for ind, i in enumerate(hvoting.BERT_Predictions):
    for j, n in enumerate(i):
        if n >= 0.5:
            i[j] = 1
        else:
            i[j] = 0
        
for ind, i in enumerate(hvoting.GRU_Predictions):
    for j, n in enumerate(i):
        if n >= 0.5:
            i[j] = 1
        else:
            i[j] = 0

for ind, i in enumerate(hvoting.MLP_Predictions):
    for j, n in enumerate(i):
        if n >= 0.5:
            i[j] = 1
        else:
            i[j] = 0

for ind, i in enumerate(hvoting.LSTM_Predictions):
    for j, n in enumerate(i):
        if n >= 0.5:
            i[j] = 1
        else:
            i[j] = 0

In [6]:
from operator import add
hvoting["Ensemble_predictions"] = ""
hvoting['Ensemble_predictions'] = hvoting['Ensemble_predictions'].astype(object)


for ind in hvoting.index:
    x = list( map(add, hvoting['CNN_Predictions'][ind], hvoting['BERT_Predictions'][ind] ))
    x2 = list( map(add, x, hvoting['GRU_Predictions'][ind] ))
    x3 = list( map(add, x2, hvoting['MLP_Predictions'][ind] ))
    x4 = list( map(add, x3, hvoting['LSTM_Predictions'][ind] ))
    hvoting.at[ind, 'Ensemble_predictions'] = x4

In [7]:
for ind, i in enumerate(hvoting.Ensemble_predictions):
    for j, n in enumerate(i):
        if n >= 3:
            i[j] = 1
        else:
            i[j] = 0

In [ ]:
y_test = hvoting.target_list.tolist()
y_test = np.asarray(y_test, dtype=np.int64)
y_pred = hvoting.Ensemble_predictions.tolist()

#Metrics 
import sklearn.metrics

def Accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if (sum(np.logical_and(y_true[i], y_pred[i])) != 0) & (sum(np.logical_or(y_true[i], y_pred[i])) != 0):
            temp += (sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i])))
    return temp / y_true.shape[0]

def Loss_Metric(y_true, y_pred):
    return np.any(y_true != y_pred, axis=1).mean()
    
#MR = np.all(y_pred_class_rfc == y_test, axis=1).mean()
#Loss_Metric = np.any(y_test != y_pred_class_rfc, axis=1).mean()

print('Exact Match Ratio: {0}'.format(sklearn.metrics.accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)))

print('Hamming loss: {0}'.format(sklearn.metrics.hamming_loss(y_test, y_pred))) 

print('Accuracy: {0}'.format(Accuracy(y_test, y_pred)))

print('Recall: {0}'.format(sklearn.metrics.precision_score(y_test, y_pred, average='samples'))) 

print('Precision: {0}'.format(sklearn.metrics.recall_score(y_test, y_pred, average='samples')))

print('F1 Measure: {0}'.format(sklearn.metrics.f1_score(y_test, y_pred, average='samples'))) 

# Soft Voting

In [9]:
svoting = df.copy()

for index, i in enumerate(svoting.CNN_Predictions):
        svoting.CNN_Predictions[index] = [j * 0 for j in i]
        
for index, i in enumerate(svoting.MLP_Predictions):
        svoting.MLP_Predictions[index] = [j * 70 for j in i]
        
for index, i in enumerate(svoting.BERT_Predictions):
        svoting.BERT_Predictions[index] = [j * 4 for j in i]        
        
for index, i in enumerate(svoting.GRU_Predictions):
        svoting.GRU_Predictions[index] = [j * 20 for j in i]
        
for index, i in enumerate(svoting.LSTM_Predictions):
        svoting.LSTM_Predictions[index] = [j * 5 for j in i]

In [10]:
from operator import add
svoting["Ensemble_predictions"] = ""
svoting['Ensemble_predictions'] = svoting['Ensemble_predictions'].astype(object)


for ind in svoting.index:
    x = list( map(add, svoting['CNN_Predictions'][ind], svoting['MLP_Predictions'][ind] ))
    x2 = list( map(add, x, svoting['BERT_Predictions'][ind] ))
    x3 = list( map(add, x2, svoting['GRU_Predictions'][ind] ))
    x4 = list( map(add, x3, svoting['LSTM_Predictions'][ind] ))
    svoting.at[ind, 'Ensemble_predictions'] = x4

In [11]:
#Replace all values >= 1 to 1 to make the list binary for calculating the accuracy
for ind, i in enumerate(svoting.Ensemble_predictions):
    for j, n in enumerate(i):
        if n >= 26:
            i[j] = 1
        else:
            i[j] = 0

In [ ]:
y_test = svoting.target_list.tolist()
y_test = np.asarray(y_test, dtype=np.int64)
y_pred = svoting.Ensemble_predictions.tolist()


#Metrics 

import sklearn.metrics

print('Exact Match Ratio: {0}'.format(sklearn.metrics.accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)))

print('Hamming loss: {0}'.format(sklearn.metrics.hamming_loss(y_test, y_pred))) 

print('Accuracy: {0}'.format(Accuracy(y_test, y_pred)))

print('Recall: {0}'.format(sklearn.metrics.precision_score(y_test, y_pred, average='samples'))) 

print('Precision: {0}'.format(sklearn.metrics.recall_score(y_test, y_pred, average='samples')))

print('F1 Measure: {0}'.format(sklearn.metrics.f1_score(y_test, y_pred, average='samples'))) 

# Stacking

In [ ]:
stacking = df.copy()
stacking.head(1)

In [55]:
for ind, j in enumerate(stacking.index):
    for index, i in enumerate(stacking.CNN_Predictions[j]):
        x = "CNN" +str(index)
        stacking.at[ind, x] = i
        
    for index, i in enumerate(stacking.RF_Predictions[j]):
        x = "RF" +str(index)
        stacking.at[ind, x] = i
        
    for index, i in enumerate(stacking.BERT_Predictions[j]):
        x = "BERT" +str(index)
        stacking.at[ind, x] = i
        
    for index, i in enumerate(stacking.GRU_Predictions[j]):
        x = "GRU" +str(index)
        stacking.at[ind, x] = i
        
    for index, i in enumerate(stacking.LSTM_Predictions[j]):
        x = "LSTM" +str(index)
        stacking.at[ind, x] = i
        
    for index, i in enumerate(stacking.target_list[j]):
        x = "Y" +str(index)
        stacking.at[ind, x] = i

stacking = stacking.drop(['CNN_Predictions', 'RF_Predictions', 'BERT_Predictions','GRU_Predictions', 'LSTM_Predictions',  'target_list', 'TITLE'], axis=1)

In [56]:
X = stacking.drop(['Y0', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9', 'Y10', 'Y11', 'Y12', 'Y13'], axis=1)
y = stacking[['Y0', 'Y1', 'Y2', 'Y3', 'Y4', 'Y5', 'Y6', 'Y7', 'Y8', 'Y9', 'Y10', 'Y11', 'Y12', 'Y13']]

In [57]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [58]:
y_train = np.asarray(y_train, dtype=np.int64)
y_test = np.asarray(y_test, dtype=np.int64)

In [ ]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier
import time
zeitanfang = time.time()

rf = RandomForestClassifier(random_state = 42)
#Hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 300, cv = 3, verbose=2, random_state=42, n_jobs = -1)


rf_random.fit(X_train, y_train)

zeitende = time.time()
print("Dauer Programmausführung:",)
print(zeitende-zeitanfang)

In [60]:
y_pred_class_rfc = rf_random.predict(X_test)

In [ ]:
y_test = y_test.tolist()
y_test = np.asarray(y_test, dtype=np.int64)
y_pred = y_pred_class_rfc.tolist()

#Metrics 
import sklearn.metrics

def Accuracy(y_true, y_pred):
    temp = 0
    for i in range(y_true.shape[0]):
        if (sum(np.logical_and(y_true[i], y_pred[i])) != 0) & (sum(np.logical_or(y_true[i], y_pred[i])) != 0):
            temp += (sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i])))
    return temp / y_true.shape[0]

def Loss_Metric(y_true, y_pred):
    return np.any(y_true != y_pred, axis=1).mean()


print('Exact Match Ratio: {0}'.format(sklearn.metrics.accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)))

print('Hamming loss: {0}'.format(sklearn.metrics.hamming_loss(y_test, y_pred))) 

print('Accuracy: {0}'.format(Accuracy(y_test, y_pred)))

print('Recall: {0}'.format(sklearn.metrics.precision_score(y_test, y_pred, average='samples'))) 

print('Precision: {0}'.format(sklearn.metrics.recall_score(y_test, y_pred, average='samples')))

print('F1 Measure: {0}'.format(sklearn.metrics.f1_score(y_test, y_pred, average='samples'))) 